In [2]:
import json
import pprint
from collections import Counter
from operator import itemgetter
import csv
from dotmap import DotMap
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import openpyxl

In [3]:
file = 'geovistory_query_exports/nb_social_relationship_20220512.json'

with open(file, encoding='utf-8') as json_file:
    data_srel = json.load(json_file)

dmsr = DotMap(data_srel)
dmsr_r = dmsr.rows

In [4]:
# pulizzia delle stringhe

def clean_up_str(string):
    a = ''
    if string:
        a = string.replace('\xa0', '').replace('  ', ' ').strip()
    return a  


# Funzione di elminazione delle dupplicazioni in leggenda
##  https://stackoverflow.com/questions/19385639/duplicate-items-in-legend-in-matplotlib

def legend_without_duplicate_labels(ax):
    handles, labels = ax.get_legend_handles_labels()
    unique = [(h, l) for i, (h, l) in enumerate(zip(handles, labels)) if l not in labels[:i]]
    ax.legend(*zip(*unique))

# calcolo dell'anno (medio)

def date(r_col):

    lista=[]
    for v in r_col['values']:
        if v.value.timePrimitive:
            lista.append(int(v.value.timePrimitive.label[:4]))
            
    if len(lista)>=1:
        return int(np.rint(np.mean(lista)))
    else:
        return np.nan


# sintesi time-spen (anno medio: begin, at some time, end)

def datatio(r_col_begin, r_col_someTime, r_col_end, element):

    begin, some_time, end = date(r_col_begin), date(r_col_someTime), date(r_col_end)
    element+= [begin, some_time, end]

    return(element)

In [5]:
# Da riscrivere. Il flusso può essere razionalizzato e snellito 

# funzione particolare

def integratio_sr(element):    
    if len(r.col_6.entities)==2:
        for el in r.col_6.entities:
                if (el.pk_entity) != element[1]:
                    element += ['partner', el.pk_entity, clean_up_str(el.entity_label)]
    elif len(r.col_6.entities)==1:
        if (r.col_6.entities[0].pk_entity) != element[1]:
            element += ['error', r.col_6.entities[0].pk_entity, clean_up_str(r.col_6.entities[0].entity_label)]
        else:
            if len(r.col_7.entities) >0:
                element += ['error', r.col_7.entities[0].pk_entity, clean_up_str( r.col_7.entities[0].entity_label)]
            else:
                if len(r.col_8.entities) >0:
                    element += ['error', r.col_8.entities[0].pk_entity, clean_up_str( r.col_8.entities[0].entity_label)]
                else:
                    element += ['error', np.nan, np.nan]
    elif len(r.col_6.entities) >2:
        i=0
        while i in range(len(r.col_6.entities)):
            for el in r.col_6.entities:
                if (el.pk_entity) != element[1]:
                    element += ['error', el.pk_entity, clean_up_str(el.entity_label)]
                    break
                else:
                    i+=1
            break
        if i == len(r.col_6.entities):
            element += ['error', r.col_6.entities[0].pk_entity, clean_up_str(r.col_6.entities[0].entity_label)]
    else:
        if len(r.col_7.entities)>0:
            if len(r.col_7.entities) ==1:
                if (r.col_7.entities[0].pk_entity) != element[1]:
                    element += ['target', r.col_7.entities[0].pk_entity, clean_up_str(r.col_7.entities[0].entity_label)]
                else:
                    if len(r.col_8.entities)>0:     # ridondante
                        if len(r.col_8.entities)==1:
                            if (r.col_8.entities[0].pk_entity) != element[1]:
                                element += ['source', r.col_8.entities[0].pk_entity, clean_up_str(r.col_8.entities[0].entity_label)]
                            else: 
                                element += ['error', r.col_8.entities[0].pk_entity, clean_up_str(r.col_8.entities[0].entity_label)]
                        elif len(r.col_8.entities)>1:
                            i=0
                            while i in range(len(r.col_8.entities)):
                                for el in r.col_8.entities:
                                    if (el.pk_entity) != element[1]:
                                        element += ['error', el.pk_entity, clean_up_str(el.entity_label)]
                                        break
                                    else:
                                        i+=1
                                break
                            if i == len(r.col_8.entities):
                                element += ['error', r.col_8.entities[0].pk_entity, clean_up_str(r.col_8.entities[0].entity_label)]
            elif len(r.col_7.entities) > 1:
                i=0
                while i in range(len(r.col_7.entities)):
                    for el in r.col_7.entities:
                        if (el.pk_entity) != element[1]:
                            element += ['error', el.pk_entity, clean_up_str(el.entity_label)]
                            break
                        else:
                            i+=1
                    break
                if i == len(r.col_7.entities):
                    element += ['error', r.col_7.entities[0].pk_entity, clean_up_str(r.col_7.entities[0].entity_label)]
        else:
            if len(r.col_8.entities)>1:
                i=0
                while i in range(len(r.col_8.entities)):
                    for el in r.col_8.entities:
                        if (el.pk_entity) != element[1]:
                            element += ['error', el.pk_entity, clean_up_str(el.entity_label)]
                            
                            break
                        else:
                            i+=1
                    break
                if i == len(r.col_7.entities):
                    element += ['error', r.col_8.entities[0].pk_entity, clean_up_str(r.col_8.entities[0].entity_label)]
            elif len(r.col_8.entities)==0:
                element += ['error', np.nan, np.nan]
            else:
                element += ['error', r.col_8.entities[0].pk_entity, clean_up_str(r.col_8.entities[0].entity_label)]

    element += [r.col_4.entity.pk_entity, r.col_5.entities[0].entity_label if len(r.col_5.entities)>0 else np.nan]

     

# Social Relationship list = srl

srl =[['pk_processetto', 'pk_person', 'person', 'qualità', 'ruolo', 'pk_secondPerson', 'secondPerson', 'id_relazione', 'relation_type', 'begin', 'at_some_time', 'end']]


# pk_processetto/pk_processetto(union), pk_person, person_name, qualità
## Essendo la query definita a partire dalle Social Relationship, possono essere riportati più enquiry di validita (es. quando la relazione intercorre tra due richiedenti)
for r in dmsr_r:
    i = 0
    if len(r.col_0.entities) >= 1:
        for el in r.col_0.entities:
            element = []
            element += [el.pk_entity, r.col_1.entities[i].pk_entity if len(r.col_1.entities)>i else r.col_1.entities[(i-1)].pk_entity, clean_up_str(r.col_1.entities[(i)].entity_label) if len(
                r.col_1.entities)>i else clean_up_str(r.col_1.entities[(i-1)].entity_label), 'richiedente']
            i+=1
            integratio_sr(element)
            datatio(r.col_9, r.col_10, r.col_11, element)
            srl.append(element)
# Una relazione nei casi di 'vedovanza' può essere riferita a un'enquiry per mezzo dell'union
    else:   # (if len(r.col_0.entities) == 0:)
        if len(r.col_3.entities) >= 1:
            for el in r.col_3.entities:
                element = []
                element += [el.pk_entity]
                if len(r.col_2.entities) ==1:
                    element += [r.col_2.entities[0].pk_entity, clean_up_str(r.col_2.entities[0].entity_label), 'defunto']
# A causa di un difetto della query di partenza possono figurare più defunti; in tal caso è necessario individuare il corretto soggetto della relazione
                elif len(r.col_2.entities) >1:
                    lista_soggetti = []
                    if len(r.col_6.entities)>0:
                        for el in r.col_6.entities:
                            lista_soggetti.append(el.pk_entity)
                        for e in r.col_2.entities:
                            if e.pk_entity in lista_soggetti:
                                element += [e.pk_entity, clean_up_str(e.entity_label), 'defunto']
                    else:
                        for el in r.col_7.entities:
                            lista_soggetti.append(el.pk_entity) 
                        for el in r.col_8.entities:
                            lista_soggetti.append(el.pk_entity)
                        if e.pk_entity in lista_soggetti:
                                element += [e.pk_entity, clean_up_str(e.entity_label), 'defunto']
                        else:
                            element += ['error', np.nan, np.nan]
                else:
                    if len(r.col_6.entities)>0:
                        element += [r.col_6.entities[0].pk_entity, r.col_6.entities[0].entity_label, 'error']
                    else:
                        element += [r.col_7.entities[0].pk_entity if len(r.col_7.entities)>0 else r.col_8.entities[0].pk_entity if len(
                            r.col_8.entities)>0 else np.nan, r.col_7.entities[0].entity_label if len(r.col_7.entities)>0 else r.col_8.entities[0].entity_label if len(
                                r.col_8.entities)>0 else np.nan, 'error']
                integratio_sr(element)
                datatio(r.col_9, r.col_10, r.col_11, element)
                srl.append(element)
        else:
            element = [np.nan]
            if len(r.col_6.entities)>0:
                element += [r.col_6.entities[0].pk_entity, r.col_6.entities[0].entity_label, 'altro']
            else:
                if len(r.col_7.entities)>0:
                    element += [r.col_7.entities[0].pk_entity, r.col_7.entities[0].entity_label, 'altro']
                else:
                    if len(r.col_8.entities)>0:
                        element += [r.col_8.entities[0].pk_entity, r.col_8.entities[0].entity_label, 'altro']
                    else:
                        # assenza di individui coinvolti nella relazione --> errore di compilazione
                        element += ['error', np.nan, np.nan]
            integratio_sr(element)
            datatio(r.col_9, r.col_10, r.col_11, element)
            srl.append(element)
    


df_social = pd.DataFrame(srl[1:], columns=srl[0])
df_social


,pk_processetto,pk_person,person,qualità,ruolo,pk_secondPerson,secondPerson,id_relazione,relation_type,begin,at_some_time,end
0,1006729.0,999599,"Francesco di Vincenzo Moscato (da Fabriano, te...",richiedente,partner,999827.0,"Angelo di Domenico Venturino (da Fabriano, mer...",1006504,Origine comune,NaN,NaN,NaN
1,776800.0,776682,Lorenzo di Sebastiano Collo (bellunese),richiedente,source,776696.0,"Gottardo di Pietro Blasso (da Selva di Cadore,...",1006510,Vicinanza a/frequentazione di parenti e/o affini,1567.0,NaN,NaN
2,1006729.0,999599,"Francesco di Vincenzo Moscato (da Fabriano, te...",richiedente,partner,999827.0,"Angelo di Domenico Venturino (da Fabriano, mer...",1006514,Frequentazione occasionale,NaN,NaN,NaN
3,1006729.0,999599,"Francesco di Vincenzo Moscato (da Fabriano, te...",richiedente,source,999827.0,"Angelo di Domenico Venturino (da Fabriano, mer...",1006528,Vicinanza a/frequentazione di parenti e/o affini,NaN,NaN,NaN
4,776302.0,776078,Bartolomeo Evangelista Giustinelli (da Bornato),richiedente,partner,776131.0,"Tommaso di Battista Cineis (da Bornato, murer)",1006549,"Conoscenza, frequentazione e pratica",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
9583,998913.0,998830,Gada di Comini (bergamasca),defunto,partner,998871.0,Maria Elisabetta moglie di Andrea Panigado (da...,999005,Vicinato abitativo nel paese d'origine,NaN,NaN,NaN
9584,999218.0,998838,"Andrea di Angelo Pasetto (da Magno, erbariol)",richiedente,partner,998852.0,"Innocenzo di Paolo Giorgi (da Dosso, erbariol)",999009,Parentela e/o affinità,1578.0,NaN,NaN
9585,999218.0,998838,"Andrea di Angelo Pasetto (da Magno, erbariol)",richiedente,partner,998887.0,"Giacomo di Bernardo Vivenzi (da Lavino, lavora...",999063,Parentela e/o affinità,NaN,NaN,NaN
9586,999679.0,999563,Giovanni di Antonio Maiti (da Bergamo),richiedente,partner,999580.0,"Vincenzo di Pasini Maiti (da Bergamo, mercante...",999768,Origine comune,NaN,NaN,NaN


In [6]:
df_social[df_social.relation_type.notnull() & df_social.relation_type.str.contains('Formazione professionale comune|Periodo di garzonato|Frequentazione professionale di natura sub')]

,pk_processetto,pk_person,person,qualità,ruolo,pk_secondPerson,secondPerson,id_relazione,relation_type,begin,at_some_time,end
18,1007147.0,1006856,Orsola di Angelo Collotis (da Visinale),richiedente,source,1006982.0,Giovanni Battista di Giovanni Giacomo De Bono ...,1007080,Frequentazione professionale di natura subalte...,1583.0,NaN,1588.0
39,1008665.0,1008475,"Alessandro di Lorenzo Ferrarini (da Clusone, s...",richiedente,source,1008524.0,Comin di Giovanni Bonelli (mercante da colori),1008598,Periodo di garzonato,1590.0,NaN,1597.0
46,1010029.0,1009983,"Daniele di Francesco Della Pola (da Gussago, p...",richiedente,source,1009991.0,Bartolomeo di Giovanni Francesco Mesti (bresci...,1011225,Periodo di garzonato,1588.0,NaN,1595.0
54,1011701.0,1011507,"Giovanni di Tommaso Molinari (da Talmassons, p...",richiedente,source,1011527.0,"Giuseppe di Giovanni Zorio (da Ossero, paterno...",1011613,Periodo di garzonato,1586.0,NaN,1597.0
55,1011701.0,1011507,"Giovanni di Tommaso Molinari (da Talmassons, p...",richiedente,source,1011527.0,"Giuseppe di Giovanni Zorio (da Ossero, paterno...",1011620,Frequentazione professionale di natura subalte...,1586.0,NaN,1597.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9524,991277.0,990965,Baldassarre di Agostino Marchetti (da Noventa ...,richiedente,source,990993.0,"Francesco di Antonio Armellini (da Venezia, co...",991190,Periodo di garzonato,1586.0,NaN,1591.0
9525,991277.0,990965,Baldassarre di Agostino Marchetti (da Noventa ...,richiedente,source,990993.0,"Francesco di Antonio Armellini (da Venezia, co...",991196,Frequentazione professionale di natura subalte...,1591.0,NaN,NaN
9532,991530.0,991378,Maria (teutonica),richiedente,source,991386.0,Giovanni di Matteo Faise (dalla Baviera),991695,Frequentazione professionale di natura subalte...,1594.0,NaN,NaN
9542,992246.0,992063,Giacomo di Giovanni Battista Colonne (da Canal...,richiedente,source,992093.0,"Ruggero detto di Ca’ Michiel (veneziano, scult...",992226,Periodo di garzonato,1586.0,NaN,1595.0


In [7]:
set_garzonato = df_social[df_social.relation_type.notnull() & df_social.relation_type.str.contains('Formazione professionale comune|Periodo di garzonato')]

In [8]:
lista = []
for proc in set_garzonato.pk_processetto.fillna(0):
    if proc>0:
        lista.append(proc)
    else:
        print(proc)    


print(len(set_garzonato), len(lista), len(set(lista)))
    


0.0
0.0
0.0
358 355 324


In [9]:
processetti_garzonato = set(lista)

In [10]:
processetti_garzonato = [int(e) for e in processetti_garzonato if e!= np.nan]

In [11]:
file = 'geovistory_query_exports/nb_overwiev_processetti_20220510.json'

with open(file, encoding='utf-8') as json_file:
    data = json.load(json_file)

dmData = DotMap(data)
dm_r = dmData.rows


### Processetti list = psl

psl=[['volume', 'section', 'number_archival_units', 'pk_processetto', 'enquiry', 'motivation_type',
 'start_date','end_date', 'anno', 'id_union', 'number_union']]


for r in dm_r:
    element = []
    # volume
    element += ['-'.join([v.entity_label for v in r.col_0.entities]) if len(r.col_0.entities)>=1 else np.nan]   

    # sezione (unità archivistica)
    element += [clean_up_str(r.col_1.entities[0].entity_label)]

    # numero di section per enquiry 
    element += [len(r.col_1.entities)]

    # enquiry 
    element += [r.col_2.entity.pk_entity, clean_up_str(r.col_2.entity.entity_label)]
   
    # motivation type
    element += [clean_up_str('; '.join([v.entity_label for v in r.col_3.entities])) if len(r.col_3.entities)>=1 else np.nan]

    # arco e anno processetto
    valor = []
    if len(r.col_4['values']) == 0:
        element += [np.nan, np.nan, np.nan]
    else:
        for v in  r.col_4['values']:
            valor.append(int(v.value.timePrimitive.label[:4]))
        element += [r.col_4['values'][0]['value'].timePrimitive.label[:10], r.col_4[
            'values'][len(r.col_4['values'])-1]['value'].timePrimitive.label[:10], round(np.mean(valor))]

    # union
    if len(r.col_5.entities)==1:
        element += [r.col_5.entities[0].pk_entity, len(r.col_5.entities)]
    elif len(r.col_5.entities)>1:
        element += ['-'.join([str(v.pk_entity) for v in r.col_5.entities]) if len(r.col_5.entities)>=1 else np.nan, len(r.col_5.entities)]  
    else:
        element += [np.nan, 0]
        
    psl.append(element)
    

df_proc = pd.DataFrame(psl[1:], columns=psl[0])  
df_proc[df_proc.pk_processetto.isin(processetti_garzonato)].sort_values(by='anno')

    

,volume,section,number_archival_units,pk_processetto,enquiry,motivation_type,start_date,end_date,anno,id_union,number_union
3327,EM 01,Processeto di Margherita di Matteo Paulini e d...,1,776302,Bartolomeo Evangelista Giustinelli (da Bornato),Stato libero di “foresto”,1592-09-15,1592-09-15,1592.0,NaN,0
137,EM 01,Processetto di Giovanni Maria di Bernardo Zill...,1,1057148,Giovanni Maria di Bernardo Zillotti (da Rivolt...,Stato libero di “foresto”,1592-11-14,1592-11-14,1592.0,NaN,0
132,EM 01,Processetto di Giacomo di Matteo Mometto (da T...,1,1056166,"Giacomo di Matteo Mometto (da Treviso, cimador...",Stato libero di “foresto”,1592-11-13,1592-11-13,1592.0,NaN,0
129,EM 01,Processetto di Pietro di Melchiorre (teutonico...,1,1055734,"Pietro di Melchiorre (da Badia, tesser da pann...",Stato libero di “foresto”,1592-11-11,1592-11-11,1592.0,NaN,0
3255,EM 01,Processetto di Bartolomeo di Giovanni (bergama...,1,761330,"Bartolomeo di Giovanni (bergamasco, laner)",Stato libero di “foresto”,1592-07-29,1592-07-29,1592.0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
1155,EM 25,Processetto di Giovanni di Francesco Brunelli ...,1,1509626,"Giovanni di Francesco Brunelli (da Bologna, st...",Stato libero di “foresto”,1627-08-10,1627-08-10,1627.0,NaN,0
680,EM 25,Processetto di Andrea di Antonio Martorello (d...,1,1222779,Andrea di Antonio Martorello (dalla diocesi di...,Stato libero di “foresto”,1627-06-24,1627-06-24,1627.0,NaN,0
848,EM 25,Processetto di Bernardo di Francesco Bettaroli...,2,1288009,"Bernardo di Francesco Bettaroli (bergamasco, b...",Stato libero di “foresto”,1627-07-17,1627-07-19,1627.0,NaN,0
873,EM 25,Processetto di Giovanni di Antonio (da Valle d...,1,1299237,"Giovanni di Antonio (da Valle dei Signori, tes...",Stato libero di “foresto”,1627-07-25,1627-07-25,1627.0,NaN,0


In [12]:
df_proc_garz = df_proc[df_proc.pk_processetto.isin(processetti_garzonato)  & df_proc.anno.notnull() & (df_proc.anno<1609)].sort_values(by='anno')

In [13]:
df_proc_garz

,volume,section,number_archival_units,pk_processetto,enquiry,motivation_type,start_date,end_date,anno,id_union,number_union
3290,EM 01,Processetto di Alessandro di Giacomo Vollesell...,1,768753,"Alessandro di Giacomo Vollesella (cadorino, te...",Stato libero di “foresto”,1592-08-22,1592-08-22,1592.0,NaN,0
138,EM 01,Processetti di Sebastiano di Francesco Bassi (...,1,1057367,"Sebastiano di Francesco Bassi (da Bassano, tes...",Stato libero di “foresto”,1592-11-23,1592-11-23,1592.0,NaN,0
256,EM 01,"Processetto di Giacomo Vicenti (da Vicenza, te...",1,1083057,"Giacomo Vincenti (da Vicenza, tesser da panni ...",Stato libero di “foresto”,1592-01-06,1592-01-06,1592.0,NaN,0
269,EM 01,Processetto di Giovanni Antonio di Giovanni Ma...,1,1086652,Giovanni Antonio di Giovanni Maria Valsech (le...,Stato libero di “foresto”,1592-11-01,1592-11-01,1592.0,NaN,0
3327,EM 01,Processeto di Margherita di Matteo Paulini e d...,1,776302,Bartolomeo Evangelista Giustinelli (da Bornato),Stato libero di “foresto”,1592-09-15,1592-09-15,1592.0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
1021,EM 08,Processetto di Francesco di Pietro Angelo Visc...,1,1471025,Francesco di Pietro Angelo Visconti (dalla Val...,Stato libero di “foresto”,1604-01-29,1604-01-29,1604.0,NaN,0
1250,EM 08,Processetto di Gerolamo di Antonio Tralio (dal...,1,1548615,Gerolamo di Antonio Tralio (dalla diocesi di R...,Stato libero di “foresto”,1604-04-24,1604-04-24,1604.0,NaN,0
1277,EM 08,Processetto Bartolomeo di Giovanni Maria (dall...,1,1553948,Bartolomeo di Giovanni Maria (dalla Riviera di...,Stato libero di “foresto”,1604-04-27,1604-04-27,1604.0,NaN,0
1160,EM 08,Processetto di Gerolamo di Alessandro Bozzato ...,1,1510393,"Gerolamo di Alessandro Bozzato (da Angarano, l...",Stato libero di “foresto”,1604-03-14,1604-03-14,1604.0,NaN,0


In [14]:
df_proc_garz = df_proc_garz[['volume', 'section', 'number_archival_units', 'pk_processetto',
       'enquiry', 'motivation_type', 'start_date', 'end_date', 'anno']]

In [15]:
mot_tyoe_garz = set_garzonato[['pk_processetto','relation_type',
       'begin', 'at_some_time', 'end']]

In [16]:
df_proc_garz = df_proc_garz.merge(mot_tyoe_garz, on='pk_processetto', how='left')

In [17]:
# df_proc_garz.to_excel('processetti_garzonato.xlsx', index=False)